<a href="https://colab.research.google.com/github/chi-hun/dacon/blob/main/2023_%EA%B5%90%EC%9B%90%EA%B7%B8%EB%A3%B9_AI_%EC%B1%8C%EB%A6%B0%EC%A7%80(RCNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2023 교원그룹 AI 챌린지

https://dacon.io/competitions/official/236042/overview/description

In [116]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import albumentations as A

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/데이콘/교원/open.zip'

In [83]:
train_csv = pd.read_csv('/content/train.csv')
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76888 entries, 0 to 76887
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        76888 non-null  object
 1   img_path  76888 non-null  object
 2   label     76888 non-null  object
dtypes: object(3)
memory usage: 1.8+ MB


In [84]:
train_csv

,id,img_path,label
0,TRAIN_00000,./train/TRAIN_00000.png,빨간색
1,TRAIN_00001,./train/TRAIN_00001.png,머
2,TRAIN_00002,./train/TRAIN_00002.png,차차
3,TRAIN_00003,./train/TRAIN_00003.png,써
4,TRAIN_00004,./train/TRAIN_00004.png,놓치다
...,...,...,...
76883,TRAIN_76883,./train/TRAIN_76883.png,회
76884,TRAIN_76884,./train/TRAIN_76884.png,겪다
76885,TRAIN_76885,./train/TRAIN_76885.png,벨트
76886,TRAIN_76886,./train/TRAIN_76886.png,톼


In [87]:
train_csv['img_path'] = '/content/train/' + train_csv['img_path'].str.split('/').str[-1]

In [45]:
img_size = (64,103)

In [138]:
agu = A.Compose([A.RandomBrightnessContrast(always_apply=True)])
class make_dataloder(tf.keras.utils.Sequence):
    def __init__(self, path, label=None, img_size =(64,64), batch_size=32, shuffle=False, agu=None):
        self.path = path
        self.label = label
        self.img_size = img_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.agu = agu

        self.one_epoch_end()

    def __len__(self):
        a = np.ceil(len(self.path) / self.batch_size).astype(np.int32)
        return a

    def __getitem__(self, index):
        path_sl = self.path[self.data_idx[index * self.batch_size : (index + 1) * self.batch_size]]
        if self.label is not None:
            label_sl = self.label[self.data_idx[index * self.batch_size : (index + 1) * self.batch_size]]
        
        images = np.zeros((len(path_sl), self.img_size[0], self.img_size[1], 3), dtype=np.float32)
        for i, path in enumerate(path_sl):
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.img_size)

            if self.agu is not None:
                img = self.agu(image = img)['image']
            images[i] = img / 255

        if self.label is not None:
            return images, label_sl
        else:
            return images

    def one_epoch_end(self):
        self.data_idx = np.arange(len(self.path))
        if self.shuffle:
            np.random.shuffle(self.data_idx)

In [139]:
aaa = make_dataloder(train_csv['img_path'].values, train_csv['label'].values, shuffle=True, agu=agu)

In [153]:
train_gt = [gt for gt in train_csv['label']]
train_gt = "".join(train_gt)
letters = sorted(list(set(list(train_gt))))
# print(len(letters))

In [158]:
vocabulary = ["-"] + letters
idx2char = {k:v for k,v in enumerate(vocabulary, start=0)}
char2idx = {v:k for k,v in idx2char.items()}

In [161]:
def crnn():
    """
    The original feature extraction structure from CRNN paper.
    Related paper: https://ieeexplore.ieee.org/abstract/document/7801919
    """
    inputs = layers.Input(shape=(64,64,3))
    x = layers.Conv2D(
        64, 3, padding='same', activation='relu', name='conv1')(inputs)
    x = layers.MaxPool2D(pool_size=2, padding='same', name='pool1')(x)

    x = layers.Conv2D(
        128, 3, padding='same', activation='relu', name='conv2')(x)
    x = layers.MaxPool2D(pool_size=2, padding='same', name='pool2')(x)

    x = layers.Conv2D(256, 3, padding='same', use_bias=False, name='conv3')(x)
    x = layers.BatchNormalization(name='bn3')(x)
    x = layers.Activation('relu', name='relu3')(x)
    x = layers.Conv2D(
        256, 3, padding='same', activation='relu', name='conv4')(x)
    x = layers.MaxPool2D(
        pool_size=2, strides=(2, 1), padding='same', name='pool4')(x)

    x = layers.Conv2D(512, 3, padding='same', use_bias=False, name='conv5')(x)
    x = layers.BatchNormalization(name='bn5')(x)
    x = layers.Activation('relu', name='relu5')(x)
    x = layers.Conv2D(
        512, 3, padding='same', activation='relu', name='conv6')(x)
    x = layers.MaxPool2D(
        pool_size=2, strides=(2, 1), padding='same', name='pool6')(x)

    x = layers.Conv2D(512, 2, use_bias=False, name='conv7')(x)
    x = layers.BatchNormalization(name='bn7')(x)
    x = layers.Activation('relu', name='relu7')(x)

    x = layers.Reshape((-1, 512), name='reshape7')(x)

    x = layers.Bidirectional(
        layers.LSTM(units=256, return_sequences=True), name='bi_lstm1')(x)
    x = layers.Bidirectional(
        layers.LSTM(units=256, return_sequences=True), name='bi_lstm2')(x)
    outputs = layers.Dense(units=10, name='logits')(x)

    model = tf.keras.Model(inputs, outputs)

    return model

In [162]:
model = crnn()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 64, 64, 64)        1792      
                                                                 
 pool1 (MaxPooling2D)        (None, 32, 32, 64)        0         
                                                                 
 conv2 (Conv2D)              (None, 32, 32, 128)       73856     
                                                                 
 pool2 (MaxPooling2D)        (None, 16, 16, 128)       0         
                                                                 
 conv3 (Conv2D)              (None, 16, 16, 256)       294912    
                                                                 
 bn3 (BatchNormalization)    (None, 16, 16, 256)       1024  

In [163]:
model(aaa[0][0])

<tf.Tensor: shape=(32, 45, 10), dtype=float32, numpy=
array([[[ 3.08469422e-02, -1.85885802e-02, -2.39095581e-03, ...,
         -3.33153224e-03,  1.02400202e-02, -3.23552359e-03],
        [ 3.27944383e-02, -1.55929886e-02, -2.73754122e-03, ...,
         -3.69942281e-03,  1.00246677e-02, -2.69311760e-03],
        [ 3.47945616e-02, -1.29925124e-02, -3.09068803e-03, ...,
         -3.22638336e-03,  9.43937246e-03, -2.37013539e-03],
        ...,
        [ 2.19405945e-02,  1.92737356e-02,  6.47857320e-03, ...,
          1.22570470e-02,  1.32396817e-04, -5.17275603e-03],
        [ 1.93872266e-02,  1.99274495e-02,  6.16371352e-03, ...,
          1.25571871e-02, -3.44538596e-04, -5.88360755e-03],
        [ 1.66654699e-02,  1.92214698e-02,  5.34242485e-03, ...,
          1.25983534e-02,  1.05376588e-04, -6.38069585e-03]],

       [[ 3.03572062e-02, -1.86295081e-02, -2.61317776e-03, ...,
         -3.36347823e-03,  1.00063402e-02, -2.68973084e-03],
        [ 3.21194641e-02, -1.54256709e-02, -3.106